# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [2]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [3]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [4]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['p', 't', 'n', 'T', ':']
44


In [5]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [6]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [7]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [8]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [9]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [10]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([15, 43, 11,  3, 23, 19,  2, 33, 26,  6])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [11]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [12]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[15, 43, 11,  3, 23, 19,  2, 33],
        [43, 11,  3, 23, 19,  2, 33, 26],
        [11,  3, 23, 19,  2, 33, 26,  6],
        [ 3, 23, 19,  2, 33, 26,  6, 23],
        [23, 19,  2, 33, 26,  6, 23,  8]])
y training data
tensor([[43, 11,  3, 23, 19,  2, 33, 26],
        [11,  3, 23, 19,  2, 33, 26,  6],
        [ 3, 23, 19,  2, 33, 26,  6, 23],
        [23, 19,  2, 33, 26,  6, 23,  8],
        [19,  2, 33, 26,  6, 23,  8, 37]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [13]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [14]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[-0.8979, -0.6625,  0.0417, -0.4790,  1.1415, -0.0312,  1.4496,
           1.9945],
         [-1.2572,  0.4126,  1.4288, -1.0449,  0.4771,  0.0497, -0.1366,
          -0.3072],
         [-1.2769, -0.0134,  1.1682,  0.8938,  1.5531, -1.4805,  2.1914,
          -2.8512],
         [ 0.9777,  0.9142, -1.1356,  0.0538,  1.7483, -0.2650,  1.6982,
           0.9723]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [15]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [16]:
x_embeddings = embeddings_table(x_batch)

In [17]:
# just one example
x_embeddings[:1]

tensor([[[-0.2858,  0.3046,  0.9892,  0.4319,  0.5329,  0.0913, -0.2983,
           0.7466],
         [-0.3561,  0.0636, -0.2214,  0.0338,  0.1802,  1.9247, -0.6023,
           0.2158],
         [ 0.3188,  1.4802, -0.3262, -0.0736,  0.3622,  1.8036, -0.0415,
          -0.3600],
         [ 0.9777,  0.9142, -1.1356,  0.0538,  1.7483, -0.2650,  1.6982,
           0.9723],
         [ 0.1271,  1.4402,  0.7840, -0.8446,  1.4553,  0.9799, -1.4546,
          -1.3262],
         [-0.7565,  0.0149, -0.2057,  0.2637, -0.4541, -0.8826,  1.4025,
           0.9973],
         [-1.2769, -0.0134,  1.1682,  0.8938,  1.5531, -1.4805,  2.1914,
          -2.8512],
         [-0.9765,  1.3118, -1.5572, -1.4263,  1.6716,  1.1448,  0.1623,
           1.4389]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [18]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [19]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [20]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [21]:
W_q.weight.shape

torch.Size([4, 8])

In [22]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [23]:
W_q.weight@x_embeddings_total[0][0]

tensor([-0.9900,  0.1957, -0.5082,  0.8806], grad_fn=<MvBackward0>)

In [24]:
W_q(x_embeddings_total)

tensor([[[-9.9000e-01,  1.9571e-01, -5.0815e-01,  8.8062e-01],
         [ 1.6196e+00, -1.3163e-01,  7.3892e-01,  8.1585e-01],
         [ 6.5555e-01, -5.2540e-01, -1.1012e-03,  1.1084e+00],
         [ 1.1798e+00,  5.9320e-01,  1.9798e+00, -1.1220e-01],
         [ 1.3775e+00, -3.6792e-01, -7.5438e-01,  1.4883e+00],
         [-5.2437e-01, -1.8156e-01, -4.9120e-01, -2.7889e-01],
         [-6.7334e-01,  6.7639e-01, -1.4096e+00, -2.4214e-01],
         [ 2.3384e-01,  4.9549e-01,  4.0713e-01,  8.7705e-01]],

        [[-3.1642e-01,  7.4578e-02, -3.1656e-02,  6.7063e-01],
         [ 2.1488e+00, -5.9727e-01,  1.1710e+00,  8.5421e-01],
         [ 2.0982e-01,  3.6170e-01,  5.6467e-01,  1.2361e-01],
         [ 1.6318e+00, -4.0960e-01,  7.1636e-01,  1.4433e+00],
         [-2.5875e-02,  4.8284e-01, -6.1153e-01,  8.7302e-02],
         [ 2.7663e-01, -2.7599e-01, -2.2256e+00, -1.4050e-01],
         [-2.7752e-01,  8.2488e-01,  1.6303e+00,  5.0245e-01],
         [ 1.0947e-01,  6.3758e-01, -1.1488e+00, -3.6

In [25]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [26]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [27]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [28]:
print(Q[0][:3])
print(K[0][:3])

tensor([[-9.9000e-01,  1.9571e-01, -5.0815e-01,  8.8062e-01],
        [ 1.6196e+00, -1.3163e-01,  7.3892e-01,  8.1585e-01],
        [ 6.5555e-01, -5.2540e-01, -1.1012e-03,  1.1084e+00]],
       grad_fn=<SliceBackward0>)
tensor([[-0.1897, -0.0187,  0.7887, -0.7028],
        [-0.8338, -1.3802,  0.5087,  0.9066],
        [-0.5615,  0.4368,  1.5863,  0.3638]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [29]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [30]:
attention_matrix[0]

tensor([[-0.8355,  1.0953,  0.1557, -1.5591,  0.5849, -1.5733, -1.7532, -1.5589],
        [-0.2955, -0.0532,  0.5020,  1.3400, -1.0174,  1.6505,  1.3475,  1.0004],
        [-0.8944,  1.1829, -0.1961,  0.1539, -0.8771,  0.4874, -1.1771, -0.2164],
        [ 1.4053, -0.8970,  2.6962,  1.8294,  0.7818,  0.6906,  3.9187,  2.5039],
        [-1.8955,  0.3248, -1.5894,  0.1416, -2.1666,  1.8586, -1.0153, -1.0853],
        [-0.0885,  0.1851, -0.6655, -0.4838,  0.0251, -0.3884, -1.0313, -0.5087],
        [-0.8265, -1.3088, -1.6506, -1.3385, -0.2691, -0.0595, -0.3285, -1.9645],
        [-0.3490,  0.1234,  1.0500, -0.1856,  0.2963, -0.2502,  0.7700, -0.1847]],
       grad_fn=<SelectBackward0>)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [31]:
print(x_batch[0])
print(y_batch[0])

tensor([15, 43, 11,  3, 23, 19,  2, 33])
tensor([43, 11,  3, 23, 19,  2, 33, 26])


In [32]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [33]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [34]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[-0.8355,  1.0953,  0.1557, -1.5591,  0.5849, -1.5733, -1.7532, -1.5589],
        [   -inf, -0.0532,  0.5020,  1.3400, -1.0174,  1.6505,  1.3475,  1.0004],
        [   -inf,    -inf, -0.1961,  0.1539, -0.8771,  0.4874, -1.1771, -0.2164],
        [   -inf,    -inf,    -inf,  1.8294,  0.7818,  0.6906,  3.9187,  2.5039],
        [   -inf,    -inf,    -inf,    -inf, -2.1666,  1.8586, -1.0153, -1.0853],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.3884, -1.0313, -0.5087],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3285, -1.9645],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1847]],
       grad_fn=<MaskedFillBackward0>)

In [35]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[-0.8355,  1.0953,  0.1557, -1.5591,  0.5849, -1.5733, -1.7532, -1.5589],
        [   -inf, -0.0532,  0.5020,  1.3400, -1.0174,  1.6505,  1.3475,  1.0004],
        [   -inf,    -inf, -0.1961,  0.1539, -0.8771,  0.4874, -1.1771, -0.2164],
        [   -inf,    -inf,    -inf,  1.8294,  0.7818,  0.6906,  3.9187,  2.5039],
        [   -inf,    -inf,    -inf,    -inf, -2.1666,  1.8586, -1.0153, -1.0853],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.3884, -1.0313, -0.5087],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3285, -1.9645],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1847]],
       grad_fn=<SelectBackward0>)

In [36]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [37]:
masked_attention_matrix[0]

tensor([[-0.8355,  1.0953,  0.1557, -1.5591,  0.5849, -1.5733, -1.7532, -1.5589],
        [   -inf, -0.0532,  0.5020,  1.3400, -1.0174,  1.6505,  1.3475,  1.0004],
        [   -inf,    -inf, -0.1961,  0.1539, -0.8771,  0.4874, -1.1771, -0.2164],
        [   -inf,    -inf,    -inf,  1.8294,  0.7818,  0.6906,  3.9187,  2.5039],
        [   -inf,    -inf,    -inf,    -inf, -2.1666,  1.8586, -1.0153, -1.0853],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -0.3884, -1.0313, -0.5087],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.3285, -1.9645],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1847]],
       grad_fn=<SelectBackward0>)

In [38]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000, 0.7592, 0.3208, 0.0184, 0.3684, 0.0129, 0.0031, 0.0118],
        [0.0000, 0.2408, 0.4536, 0.3342, 0.0742, 0.3229, 0.0686, 0.1521],
        [0.0000, 0.0000, 0.2257, 0.1021, 0.0854, 0.1009, 0.0055, 0.0451],
        [0.0000, 0.0000, 0.0000, 0.5453, 0.4486, 0.1237, 0.8972, 0.6842],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0235, 0.3976, 0.0065, 0.0189],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0420, 0.0064, 0.0336],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0128, 0.0078],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0465]],
       grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``E_updated[0] = V[0] + E[0]`` need to think more about it

but according to my understanding `V[0]` is same as the absolute embedding of the token so `V[0] + E[0]` would be pracitically `2*absolute_mebdding[0]` since even `E[0]` was created any info about token after that so just itself, but since we don layer norm `norm(2*absolute_embedding[0]) = absolute_embedding[0]` so even if it scales a lot layer normalization brings down the scale of embedding vector so it fits with rest of embedding vector's scale

In [39]:
batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

In [40]:
batch_attention_matrix[0]

tensor([[1.0000, 0.7592, 0.3208, 0.0184, 0.3684, 0.0129, 0.0031, 0.0118],
        [0.0000, 0.2408, 0.4536, 0.3342, 0.0742, 0.3229, 0.0686, 0.1521],
        [0.0000, 0.0000, 0.2257, 0.1021, 0.0854, 0.1009, 0.0055, 0.0451],
        [0.0000, 0.0000, 0.0000, 0.5453, 0.4486, 0.1237, 0.8972, 0.6842],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0235, 0.3976, 0.0065, 0.0189],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0420, 0.0064, 0.0336],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0128, 0.0078],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0465]],
       grad_fn=<SelectBackward0>)

In [41]:
V[0]

tensor([[ 0.9657, -1.0953,  0.5170, -0.7725,  0.5741, -0.3655,  0.8440, -1.3519],
        [-0.3147,  0.6255,  0.0512,  0.4622,  0.5038,  0.3524,  0.3912,  2.1814],
        [ 0.0389,  0.4412,  2.2621, -0.3668,  1.7100,  1.0662, -0.3138,  1.7839],
        [ 0.0595, -0.7484,  0.4376,  1.2362,  0.7058,  2.0888, -0.3751,  1.9373],
        [ 0.0351,  0.1322,  1.1411,  1.5527,  1.2162,  0.2321, -0.4218,  1.5293],
        [-1.9080,  0.9093, -0.9993, -1.0242, -1.7255, -0.5056, -0.6005, -0.2389],
        [ 0.1276,  0.1237,  1.1501,  0.5791,  0.1444,  0.8994, -0.7291,  0.4171],
        [ 0.2773, -1.1045,  0.8374,  0.1178,  0.7766,  0.8175,  0.4226,  0.4561]],
       grad_fn=<SelectBackward0>)

In [42]:
batch_attention_matrix[0]@V[0]

tensor([[ 7.3247e-01, -4.4493e-01,  1.7105e+00,  4.5517e-02,  1.9536e+00,
          3.7398e-01,  8.7304e-01,  1.4791e+00],
        [-6.0083e-01,  2.4449e-01,  1.1528e+00,  2.0023e-01,  7.9385e-01,
          1.3066e+00, -3.8442e-01,  2.1162e+00],
        [-1.6150e-01,  7.7117e-02,  5.9573e-01,  8.1135e-02,  4.2341e-01,
          4.6440e-01, -1.9067e-01,  7.2963e-01],
        [ 1.1653e-01, -8.8099e-01,  2.2316e+00,  1.8441e+00,  1.3778e+00,
          2.5468e+00, -8.3303e-01,  2.3990e+00],
        [-7.5175e-01,  3.4459e-01, -3.4727e-01, -3.6474e-01, -6.4187e-01,
         -1.7431e-01, -2.4539e-01, -4.7704e-02],
        [-7.0060e-02,  1.8563e-03, -6.5275e-03, -3.5405e-02, -4.5489e-02,
          1.1963e-02, -1.5659e-02,  7.9524e-03],
        [ 3.8137e-03, -7.0757e-03,  2.1329e-02,  8.3563e-03,  7.9442e-03,
          1.7955e-02, -6.0427e-03,  8.9302e-03],
        [ 1.2898e-02, -5.1365e-02,  3.8945e-02,  5.4800e-03,  3.6115e-02,
          3.8018e-02,  1.9652e-02,  2.1212e-02]], grad_fn=<MmBack

In [43]:
V_prime = batch_attention_matrix@V

In [44]:
embedding_unnormalized = V_prime + x_embeddings_total

# Layer normalization

we just normalize across the embedding dimension to make sure this addition does not scale the embedding vector to much across each layer so that it has the scale of the original embedding vector

In [45]:
mean_emebdding = embedding_unnormalized.mean(dim=-1)
mean_emebdding.shape

torch.Size([5, 8])

In [46]:
std_embedding = embedding_unnormalized.std(dim=-1)
std_embedding.shape

torch.Size([5, 8])

In [47]:
embedding_unnormalized.shape

torch.Size([5, 8, 8])

In [48]:
print(mean_emebdding[0][0], std_embedding[0][0])

tensor(1.5204, grad_fn=<SelectBackward0>) tensor(1.5279, grad_fn=<SelectBackward0>)


In [49]:
(embedding_unnormalized[0][0] - mean_emebdding[0][0])/(std_embedding[0][0] + 0.0001)

tensor([-0.2576, -1.4224,  1.6119, -0.3143,  0.6499, -0.6753, -0.5871,  0.9948],
       grad_fn=<DivBackward0>)

In [50]:
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast
embedding_normalized[0][0]

tensor([-0.2576, -1.4224,  1.6119, -0.3143,  0.6499, -0.6753, -0.5871,  0.9948],
       grad_fn=<SelectBackward0>)

# Multi head self attention

Functionaly what multi headed attention does is, it employs multiple smaller self attention blocks, the input to each is the full embedding vectors, and the output are vectors with a smaller dimension

```

let's say the input is [6, 6, 6]  (here 6 represents the dimension of the embedding vector which goes in, and 3 is the context length)


[6, 6, 6] -> head 1 -> [2, 2, 2]'1 (here I am just using '1 as a label)

[6, 6, 6] -> head 2 -> [2, 2, 2]'2

[6, 6, 6] -> head 3 -> [2, 2, 2]'3


concat([2, 2, 2]'1, [2, 2, 2]'2, [2, 2, 2]'3) -> [6, 6, 6]'

[6, 6, 6]' -> linear layer -> [6, 6, 6]'' --> residual connection --> [6, 6, 6]'' + [6, 6, 6] --> layer norm --> [6, 6, 6]final

```



In [51]:
# writing everything we have done in one block

# ----multi head repetition--------
d_k = d_q = 4
d_v = EMBEDDING_DIMENSION

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

V_prime = batch_attention_matrix@V
# ----multi head repetition--------

embedding_unnormalized = V_prime + x_embeddings_total

mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)

embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


In [58]:
# let me create a class out of the repeated block

class SingleHead():

    def __init__(self, d_k, d_q, d_v, input_dimension) -> None:
        
        self.W_q = nn.Linear(input_dimension, d_q, bias = False)
        self.W_k = nn.Linear(input_dimension, d_k, bias = False)
        self.W_v = nn.Linear(input_dimension, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension
        pass

    def forward(self, input_embeddings):
        Q, K, V = self.W_q(input_embeddings), self.W_k(input_embeddings), self.W_v(input_embeddings)

        attention_matrix = Q@K.transpose(-2, -1) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

        masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

        batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

        V_prime = batch_attention_matrix@V

        return V_prime

        
    

In [59]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [60]:
# since our embedding is of size 8 I am gonna create 2 heads with output dimensino 4

head1, head2 = SingleHead(2, 2, 4, 8), SingleHead(2, 2, 4, 8)

In [68]:
x_out_head_1 = head1.forward(x_embeddings_total)[0]
print(x_out_head_1)
x_out_head_2 = head2.forward(x_embeddings_total)[0]
print(x_out_head_2)
print(torch.cat([x_out_head_1, x_out_head_2], dim=-1))

tensor([[ 1.6593, -1.0272,  0.7224, -0.2799],
        [ 1.1771, -1.3472,  1.5586, -0.4564],
        [ 0.7515, -1.0119,  1.4668, -0.4905],
        [-1.3261, -0.6602,  1.6438,  0.3285],
        [ 0.1408, -0.3420,  0.5275, -0.2354],
        [-0.2074,  0.0327,  0.0524, -0.0273],
        [-0.0037, -0.0144,  0.0251, -0.0025],
        [ 0.0842, -0.2148,  0.2504,  0.0653]], grad_fn=<SelectBackward0>)
tensor([[-1.0709,  1.4418,  1.6062,  1.1070],
        [-0.9335,  0.3559,  1.2817,  1.4181],
        [-0.9848,  0.6435,  1.2915,  1.3274],
        [-0.9044,  0.6626,  0.9873,  1.0175],
        [-0.2911, -0.0926,  0.4226,  0.3255],
        [-0.4405, -0.1883,  0.2199,  0.1819],
        [-0.6129,  0.1904,  0.4199,  0.2424],
        [-0.1113,  0.0740,  0.0855,  0.0445]], grad_fn=<SelectBackward0>)
tensor([[ 1.6593, -1.0272,  0.7224, -0.2799, -1.0709,  1.4418,  1.6062,  1.1070],
        [ 1.1771, -1.3472,  1.5586, -0.4564, -0.9335,  0.3559,  1.2817,  1.4181],
        [ 0.7515, -1.0119,  1.4668, -0.4905,

In [70]:
x_out_embedding_total = torch.cat([head1.forward(x_embeddings_total), head2.forward(x_embeddings_total)], dim= -1)
x_out_embedding_total[0]

tensor([[ 1.6593, -1.0272,  0.7224, -0.2799, -1.0709,  1.4418,  1.6062,  1.1070],
        [ 1.1771, -1.3472,  1.5586, -0.4564, -0.9335,  0.3559,  1.2817,  1.4181],
        [ 0.7515, -1.0119,  1.4668, -0.4905, -0.9848,  0.6435,  1.2915,  1.3274],
        [-1.3261, -0.6602,  1.6438,  0.3285, -0.9044,  0.6626,  0.9873,  1.0175],
        [ 0.1408, -0.3420,  0.5275, -0.2354, -0.2911, -0.0926,  0.4226,  0.3255],
        [-0.2074,  0.0327,  0.0524, -0.0273, -0.4405, -0.1883,  0.2199,  0.1819],
        [-0.0037, -0.0144,  0.0251, -0.0025, -0.6129,  0.1904,  0.4199,  0.2424],
        [ 0.0842, -0.2148,  0.2504,  0.0653, -0.1113,  0.0740,  0.0855,  0.0445]],
       grad_fn=<SelectBackward0>)

In [71]:
# add and norm 

embedding_unnormalized = x_out_embedding_total + x_embeddings_total
mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


Now that I have shown functionaly how multi head attention works we need to understand what it really means

it is really like head 1 decides from 0 to 3rd direction which direction it should move in and head 2 decide from 4th to 7th which direction embedding should move in
I am not a 100% sure why we have this